In [ ]:
#default_exp pretrained_models

# Pretrained Models

> A model zoo for to download and load pretrained models.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from torch.utils.model_zoo import load_url

## Model Zoo

In [ ]:
#export
models = {} 

In [ ]:
#export
def get_model(model_name: str):
    """Downloads and builds pretrained model."""
    pass

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.ipynb.
Converted 01_dataset.ipynb.
Converted 02_evaluate.ipynb.
Converted 03_train.ipynb.
Converted 04_generate_pseudo_labels.ipynb.
Converted 05_self_knowledge_distillation.ipynb.
Converted 06_create_folds.ipynb.
Converted 07_pretrained_models.ipynb.
Converted config.ipynb.
Converted index.ipynb.
